# Text Translation and Sentiment Analysis using Transformers

## Project Overview:

The objective of this project is to analyze the sentiment of movie reviews in three different languages - English, French, and Spanish. 


In [1]:
# imports
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from transformers import pipeline
import torch

### Get data from `.csv` files and then preprocess data

In [ ]:

def preprocess_data() -> pd.DataFrame:
    """
    Reads movie data from .csv files, map column names, add the "Original Language" column,
    and finally concatenate in one resultant dataframe called "df".
    """
    # Read English data
    df_eng = pd.read_csv("/content/movie_reviews_eng.csv")
    df_eng = df_eng.rename(columns={
        "Movie / TV Series": "Title",
        "Review": "Review"
    })
    df_eng["Original Language"] = "en"

    # Read French data
    df_fr = pd.read_csv("/content/movie_reviews_fr.csv")
    df_fr = df_fr.rename(columns={
        "Titre": "Title",
        "Année": "Year",
        "Critiques": "Review"
    })
    df_fr["Original Language"] = "fr"

    # Read Spanish data
    df_sp = pd.read_csv("/content/movie_reviews_sp.csv")
    df_sp = df_sp.rename(columns={
        "Título": "Title",
        "Año": "Year",
        "Sinopsis": "Synopsis",
        "Críticas": "Review"
    })
    df_sp["Original Language"] = "sp"

    # Select only the required columns and concatenate
    required_columns = ["Title", "Year", "Synopsis", "Review", "Original Language"]
    df_eng = df_eng[required_columns]
    df_fr = df_fr[required_columns]
    df_sp = df_sp[required_columns]

    df = pd.concat([df_eng, df_fr, df_sp], ignore_index=True)
    return df

df = preprocess_data()

In [ ]:
df.sample(10)

### Text translation

Translate the **Review** and **Synopsis** column values to English.

In [ ]:
# load translation models and tokenizers
fr_en_model_name = "Helsinki-NLP/opus-mt-fr-en"
es_en_model_name = "Helsinki-NLP/opus-mt-es-en"

fr_en_model = MarianMTModel.from_pretrained(fr_en_model_name)
es_en_model = MarianMTModel.from_pretrained(es_en_model_name)

fr_en_tokenizer = MarianTokenizer.from_pretrained(fr_en_model_name)
es_en_tokenizer = MarianTokenizer.from_pretrained(es_en_model_name)

def translate(text: str, model, tokenizer) -> str:
    """
    function to translate a text using a model and tokenizer
    """
    # encode the text using the tokenizer
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # generate the translation using the model
    with torch.no_grad():
        outputs = model.generate(**inputs)

    # decode the generated output and return the translated text
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

In [ ]:
# Translate French reviews and synopses
fr_reviews = df[df["Original Language"] == "fr"]["Review"].tolist()
fr_reviews_en = [translate(review, fr_en_model, fr_en_tokenizer) for review in fr_reviews]

fr_synopsis = df[df["Original Language"] == "fr"]["Synopsis"].tolist()
fr_synopsis_en = [translate(synopsis, fr_en_model, fr_en_tokenizer) for synopsis in fr_synopsis]

# Translate Spanish reviews and synopses
es_reviews = df[df["Original Language"] == "sp"]["Review"].tolist()
es_reviews_en = [translate(review, es_en_model, es_en_tokenizer) for review in es_reviews]

es_synopsis = df[df["Original Language"] == "sp"]["Synopsis"].tolist()
es_synopsis_en = [translate(synopsis, es_en_model, es_en_tokenizer) for synopsis in es_synopsis]

# Update dataframe with translated text
fr_indices = df[df["Original Language"] == "fr"].index
sp_indices = df[df["Original Language"] == "sp"].index

for i, idx in enumerate(fr_indices):
    df.at[idx, "Review"] = fr_reviews_en[i]
    df.at[idx, "Synopsis"] = fr_synopsis_en[i]

for i, idx in enumerate(sp_indices):
    df.at[idx, "Review"] = es_reviews_en[i]
    df.at[idx, "Synopsis"] = es_synopsis_en[i]

In [5]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language
26,Toc Toc,2017,"In this Spanish comedy, a group of people with...","""Toc Toc is a boring and unoriginal film that ...",sp
8,Solo: A Star Wars Story,2018,A young Han Solo (Alden Ehrenreich) joins a gr...,"""Dull and pointless, with none of the magic of...",en
29,El Incidente,2014,"In this Mexican horror film, a group of people...","""The Incident is a boring and frightless film ...",sp
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",fr
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp
10,La La Land,2016,This musical tells the story of a budding actr...,"""The Land is an absolutely beautiful film with...",fr
4,Inception,2010,Dom Cobb (Leonardo DiCaprio) is a skilled thie...,"""Inception is a mind-bending and visually stun...",en
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",fr
14,Le Fabuleux Destin d'Amélie Poulain,2001,This romantic comedy tells the story of Amélie...,"""The Fabulous Destiny of Amélie Poulain is an ...",fr
28,Torrente: El brazo tonto de la ley,1998,"In this Spanish comedy, a corrupt cop (played ...","""Torrente is a vulgar and offensive film that ...",sp


### Sentiment Analysis

Use HuggingFace pretrained model for sentiment analysis of the reviews. Store the sentiment result **Positive** or **Negative** in a new column titled **Sentiment** in the dataframe.

In [ ]:
# load sentiment analysis model
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
sentiment_classifier = pipeline("sentiment-analysis", model=model_name)

def analyze_sentiment(text, classifier):
    """
    function to perform sentiment analysis on a text using a model
    """
    # Clean the text by removing extra quotes
    clean_text = text.strip('"')
    result = classifier(clean_text)[0]
    return result['label']

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# perform sentiment analysis on reviews and store results in new column
df["Sentiment"] = df["Review"].apply(lambda x: analyze_sentiment(x, sentiment_classifier))

In [8]:
df.sample(10)

,Title,Year,Synopsis,Review,Original Language,Sentiment
21,La Casa de Papel,(2017-2021),This Spanish television series follows a group...,"""The Paper House is an exciting and addictive ...",sp,POSITIVE
25,Águila Roja,(2009-2016),This Spanish television series follows the adv...,"""Red Eagle is a boring and uninteresting serie...",sp,NEGATIVE
18,Les Visiteurs en Amérique,2000,In this continuation of the French comedy The ...,"""The film is a total waste of time. The jokes ...",fr,NEGATIVE
15,Le Dîner de Cons,1998,The film follows the story of a group of rich ...,"""I didn't like this movie at all. The concept ...",fr,NEGATIVE
3,The Godfather,1972,Don Vito Corleone (Marlon Brando) is the head ...,"""The Godfather is a classic movie that stands ...",en,POSITIVE
0,The Shawshank Redemption,1994,"Andy Dufresne (Tim Robbins), a successful bank...","""The Shawshank Redemption is an inspiring tale...",en,POSITIVE
24,Amores perros,2000,Three stories intertwine in this Mexican film:...,"""Amores dogs is an intense and moving film tha...",sp,POSITIVE
16,La Tour Montparnasse Infernale,2001,Two incompetent office workers find themselves...,"""I can't believe I've wasted time watching thi...",fr,NEGATIVE
9,The Island,2005,In a future where people are cloned for organ ...,"""The Island is a bland and forgettable sci-fi ...",en,NEGATIVE
19,Babylon A.D.,2008,"In the distant future, a mercenary has to esco...","""This film is a complete mess. The characters ...",fr,NEGATIVE


In [10]:
# export the results to a .csv file
df.to_csv("result/reviews_with_sentiment.csv", index=False)